In [ ]:
# 1. Load and Modify VGG16

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
vgg16 = models.vgg16(pretrained=True)

C:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [10:55<00:00, 844kB/s]


In [8]:
import os

model_dir = 'Model/'
model_filename = 'vgg16_pretrained.pth'
model_path = os.path.join(model_dir, model_filename)
torch.save(vgg16.state_dict(), model_path)
model_path

NameError: name 'vgg16' is not defined

In [4]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Step 2: Load the Pre-trained VGG16 Model

In [9]:
model_dir = 'Model/'
model_filename = 'vgg16_pretrained.pth'
model_path = os.path.join(model_dir, model_filename)

vgg = models.vgg16()
vgg.load_state_dict(torch.load(model_path))

C:\Users\HP\AppData\Local\Temp\ipykernel_4948\4254293550.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
# Step 3: Freeze Convolutional Layers

In [10]:
for param in vgg.features.parameters():
    param.requires_grad = False

In [ ]:
# Step 4: Add Custom Classifier with Trainable Parameters

In [11]:
# Modify the classifier to fit MNIST (10 classes)
vgg.classifier[6] = nn.Linear(4096, 10)

In [ ]:
# Step 5:  Train Classifier Layers on Training Data

In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.classifier.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

train_model(vgg, train_loader, criterion, optimizer, num_epochs=5)

In [ ]:
# Fine-tune Hyperparameters and Unfreeze More Layers as Needed

In [ ]:
for param in vgg.features[10:].parameters():
    param.requires_grad = True

# Re-define optimizer to include newly unfrozen parameters
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, vgg.parameters()), lr=0.0001)

# Fine-tune the model
train_model(vgg, train_loader, criterion, optimizer, num_epochs=3)

In [ ]:
# Final Evaluation

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total}%")

evaluate_model(vgg, test_loader)